In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from random import random

# Paso 1: Cargar los datos
data = pd.read_csv('../dataframe/champions_23_24.csv')

# Paso 2: Definir la clase para el grafo de la cadena de Markov
class MarkovChainGraphUndirectedUnweighted:
    def __init__(self):
        self.connections = {}

    def add_node(self, node: str) -> None:
        self.connections[node] = {}

    def add_transition_probability(
        self, node1: str, node2: str, probability: float
    ) -> None:
        if node1 not in self.connections:
            self.add_node(node1)
        if node2 not in self.connections:
            self.add_node(node2)
        self.connections[node1][node2] = probability

    def get_nodes(self) -> list[str]:
        return list(self.connections)

    def transition(self, node: str) -> str:
        current_probability = 0
        random_value = random()

        for dest in self.connections[node]:
            current_probability += self.connections[node][dest]
            if current_probability > random_value:
                return dest
        return ""

In [3]:
# Paso 3: Preprocesamiento y cálculo de probabilidades de transición
equipos = [1, 3, 14, 24]
transiciones_por_equipo = {}

for equipo in equipos:
    data_equipo = data[(data['Local'] == equipo) | (data['Visitante'] == equipo)].copy()
    data_equipo.loc[:, 'Estado'] = np.where(data_equipo['Local'] == equipo, 'LocalGana', 
                                        np.where(data_equipo['Empate'] == 1, 'Empate', 'VisitanteGana'))

    transiciones_equipo = pd.crosstab(data_equipo['Estado'], data_equipo['Estado'].shift(-1), normalize='index')
    transiciones_por_equipo[equipo] = transiciones_equipo.fillna(0).values

# Paso 4: Ejecutar el modelo Markov Chain para cada equipo
for equipo, transiciones_equipo in transiciones_por_equipo.items():
    print(f"Matriz de Transición para el Equipo {equipo}:")
    print(pd.DataFrame(transiciones_equipo, index=['LocalGana', 'Empate', 'VisitanteGana'], columns=['LocalGana', 'Empate', 'VisitanteGana']))
    print()
    # Crear el grafo de la cadena de Markov
    graph = MarkovChainGraphUndirectedUnweighted()
    for estado_actual, transiciones in zip(['LocalGana', 'Empate', 'VisitanteGana'], transiciones_equipo):
        for estado_siguiente, probabilidad in zip(['LocalGana', 'Empate', 'VisitanteGana'], transiciones):
            graph.add_transition_probability(estado_actual, estado_siguiente, probabilidad)

    # Ejecutar el modelo Markov Chain
    visited = Counter(graph.get_nodes())
    node = 'LocalGana'
    for _ in range(5000):  # Número de pasos
        node = graph.transition(node)
        visited[node] += 1

    print("Número de veces que se visitó cada estado:")
    print(visited)
    print()


Matriz de Transición para el Equipo 1:
               LocalGana  Empate  VisitanteGana
LocalGana            0.0     1.0            0.0
Empate               0.0     0.4            0.6
VisitanteGana        0.2     0.4            0.4

Número de veces que se visitó cada estado:
Counter({'VisitanteGana': 2296, 'Empate': 2270, 'LocalGana': 437})

Matriz de Transición para el Equipo 3:
               LocalGana    Empate  VisitanteGana
LocalGana       0.000000  1.000000       0.000000
Empate          0.333333  0.333333       0.333333
VisitanteGana   0.000000  0.500000       0.500000

Número de veces que se visitó cada estado:
Counter({'Empate': 2481, 'VisitanteGana': 1695, 'LocalGana': 827})

Matriz de Transición para el Equipo 14:
               LocalGana    Empate  VisitanteGana
LocalGana       0.000000  1.000000            0.0
Empate          0.333333  0.166667            0.5
VisitanteGana   0.000000  1.000000            0.0

Número de veces que se visitó cada estado:
Counter({'Empate': 271